In [1]:
import copy

import numpy as np
import pandas as pd
from collections import Counter, OrderedDict, defaultdict
#from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB #if we'll decide to test different classifiers
#from sklearn.svm import SVC, NuSVC, LinearSVC #if we'll decide to test different classifiers
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from math import floor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import ipdb

import tqdm

# import sys
# sys.path.append('../../')

# for Kseniia's model
from cotrain import SSLTrainSet
from cotrain import BinaryCoTrainer
from dataloaders.dataloader import *

import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

#увеличим дефолтный размер графиков
from pylab import rcParams
import pandas as pd
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

import tqdm

/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df1 = pd.read_csv('results/COTRAIN_TEST3_DS456.csv', index_col = 0)
df2 = pd.read_csv('results/COTRAIN_TEST3_DS123.csv', index_col = 0)
df3 = pd.read_csv('results/logistic_CEM-test_3.csv', index_col = 0)
df4 = pd.read_csv('results/semiboost_TEST3.csv', index_col = 0)
df5 = pd.read_csv('results/MANIFOLDS_TEST3.csv', index_col = 0)
df6 = pd.read_csv('results/PMS2L_TEST3.csv', index_col = 0)

In [3]:
df5 = df5[['Algorithm', 'Dataset', 'L_Size', 'U_Size', 'T_Size', 'Accuracy']]

In [4]:
df_fin = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index = 'True')

In [5]:
df_fin[(df_fin['Dataset']== 'STL10') & (df_fin['Algorithm'] == 'co-train')]

,Accuracy,Algorithm,Dataset,L_Size,T_Size,U_Size,Unnamed: 0.1
60,0.342727,co-train,STL10,6,200,500,NaN
62,0.343409,co-train,STL10,10,200,500,NaN
64,0.386591,co-train,STL10,20,200,500,NaN
66,0.496818,co-train,STL10,50,200,500,NaN
68,0.583409,co-train,STL10,100,200,500,NaN
70,0.342727,co-train,STL10,6,200,500,NaN
72,0.343409,co-train,STL10,10,200,500,NaN
74,0.386591,co-train,STL10,20,200,500,NaN
76,0.496818,co-train,STL10,50,200,500,NaN
78,0.583409,co-train,STL10,100,200,500,NaN


In [7]:
all_datasets = ["STL10", "STL10_FFT2", "STL10_DWT2", "cifar10", "cifar10_FFT2" , "cifar10_DWT2"]
binary_list =[[1,2], [1,9]]

for data_set in all_datasets: 
        trace0 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'co-train')]['L_Size'][0:5],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'co-train')]['Accuracy'][0:5],
                            name='co-train',
                            mode='lines+markers', 
                            line={'dash': 'solid', 'color': 'green'})
        
        #binary 1
        trace1 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'co-train baseline')]['L_Size'][0:5],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'co-train baseline')]['Accuracy'][0:5],
                            name='co-train baseline',
                            mode='lines+markers', 
                            line={'dash': 'dash', 'color': 'green'})
        
        trace2 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM')]['L_Size'][0:5],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM')]['Accuracy'][0:5],
                            name='logistic-CEM',
                            mode='lines+markers', 
                            line={'dash': 'solid', 'color': 'red'})
        
        trace3 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM baseline')]['L_Size'][0:5],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM baseline')]['Accuracy'][0:5],
                            name='logistic-CEM baseline',
                            mode='lines+markers', 
                            line={'dash': 'dash', 'color': 'red'})
        
        trace4 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ILYA')]['L_Size'][0:5],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ILYA')]['Accuracy'][0:5],
                            name='SEMIBOOST',
                            mode='lines+markers', 
                            line={'dash': 'solid', 'color': 'purple'})
        
        trace5 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ILYA baseline')]['L_Size'][0:5],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ILYA baseline')]['Accuracy'][0:5],
                            name='SEMIBOOST baseline',
                            mode='lines+markers', 
                            line={'dash': 'dash', 'color': 'purple'}) 
        
        
        
        trace6 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ManifoldsOnKNN baseline')]['L_Size'][0:5],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ManifoldsOnKNN baseline')]['Accuracy'][0:5],
                            name='ManifoldsOnKNN baseline',
                            mode='lines+markers', 
                            line={'dash': 'dash', 'color': 'blue'})
        
        trace7 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ManifoldsOnKNN')]['L_Size'][0:5],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ManifoldsOnKNN')]['Accuracy'][0:5],
                            name='ManifoldsOnKNN',
                            mode='lines+markers', 
                            line={'dash': 'solid', 'color': 'blue'})
        
        trace8 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'PMS2L')]['L_Size'][0:5],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'PMS2L')]['Accuracy'][0:5],
                            name='PMS2L',
                            mode='lines+markers', 
                            line={'dash': 'solid', 'color': 'orange'})
        
        trace9 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'PMS2L baseline')]['L_Size'][0:5],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'PMS2L baseline')]['Accuracy'][0:5],
                            name='PMS2L baseline',
                            mode='lines+markers', 
                            line={'dash': 'dash', 'color': 'orange'})
        
        data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
        layout = {#'title': data_set,
                  'xaxis_title': 'Length of labeled data',
                  'yaxis_title': 'Accuracy of the algorithm',
                  'xaxis_tickvals' : df_fin['L_Size'].unique()
                 }

        fig = go.Figure(data=data, layout=layout)
        iplot(fig, show_link=False)

In [5]:
all_datasets = ["STL10", "STL10_FFT2", "STL10_DWT2", "cifar10", "cifar10_FFT2" , "cifar10_DWT2"]
binary_list =[[1,2], [1,9]]

for data_set in all_datasets: 
        trace0 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'co-train')]['L_Size'][5:10],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'co-train')]['Accuracy'][5:10],
                            name='co-train',
                            mode='lines+markers', 
                            line={'dash': 'solid', 'color': 'green'})

        trace1 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'co-train baseline')]['L_Size'][5:10],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'co-train baseline')]['Accuracy'][5:10],
                            name='co-train baseline',
                            mode='lines+markers', 
                            line={'dash': 'dash', 'color': 'green'})
        
        
        trace2 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM')]['L_Size'][5:10],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM')]['Accuracy'][5:10],
                            name='logistic-CEM',
                            mode='lines+markers', 
                            line={'dash': 'solid', 'color': 'red'})
        
        trace3 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM baseline')]['L_Size'][5:10],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM baseline')]['Accuracy'][5:10],
                            name='logistic-CEM baseline',
                            mode='lines+markers', 
                            line={'dash': 'dash', 'color': 'purple'}) 
        
        trace4 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ILYA')]['L_Size'][5:10],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ILYA')]['Accuracy'][5:10],
                            name='logistic-CEM',
                            mode='lines+markers', 
                            line={'dash': 'solid', 'color': 'purple'})
        
        trace5 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ILYA baseline')]['L_Size'][5:10],
                            y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ILYA baseline')]['Accuracy'][5:10],
                            name='logistic-CEM baseline',
                            mode='lines+markers', 
                            line={'dash': 'dash', 'color': 'red'})  
#         #binary 2
#         trace2 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'co-train baseline')]['L_Size'][5:10],
#                             y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'co-train baseline')]['Accuracy'][5:10],
#                             name='co-train baseline',
#                             mode='lines+markers', 
#                             line={'dash': 'dash', 'color': 'green'})
        
#         trace3 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM')]['L_Size'],
#                             y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM')]['Accuracy'],
#                             name='logistic-CEM',
#                             mode='lines+markers', 
#                             line={'dash': 'solid', 'color': 'red'})
        
#           trace2 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM')]['L_Size'],
#                             y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM')]['Accuracy'],
#                             name='logistic-CEM',
#                             mode='lines+markers', 
#                             line={'dash': 'solid', 'color': 'red'})
#         trace3 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM baseline')]['L_Size'],
#                             y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'logistic-CEM baseline')]['Accuracy'],
#                             name='logistic-CEM baseline',
#                             mode='lines+markers', 
#                             line={'dash': 'dash', 'color': 'red'})
        
#         trace4 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ManifoldsOnKNN baseline')]['L_Size'],
#                             y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ManifoldsOnKNN baseline')]['Accuracy'],
#                             name='ManifoldsOnKNN baseline',
#                             mode='lines+markers', 
#                             line={'dash': 'dash', 'color': 'blue'})
        
#         trace5 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ManifoldsOnKNN')]['L_Size'],
#                             y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'ManifoldsOnKNN')]['Accuracy'],
#                             name='ManifoldsOnKNN',
#                             mode='lines+markers', 
#                             line={'dash': 'solid', 'color': 'blue'})
        
#         trace6 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'PMS2L')]['L_Size'],
#                             y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'PMS2L')]['Accuracy'],
#                             name='PMS2L',
#                             mode='lines+markers', 
#                             line={'dash': 'solid', 'color': 'orange'})
        
#         trace7 = go.Scatter(x=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'PMS2L baseline')]['L_Size'],
#                             y=df_fin[(df_fin['Dataset']==data_set) & (df_fin['Algorithm'] == 'PMS2L baseline')]['Accuracy'],
#                             name='PMS2L baseline',
#                             mode='lines+markers', 
#                             line={'dash': 'dash', 'color': 'orange'})
        
        data = [trace0, trace1, trace2, trace3]
        layout = {'title': data_set,
                  'xaxis_title': 'Length of labeled data',
                  'yaxis_title': 'Accuracy of the algorithm',
                  'xaxis_tickvals' : df_fin['L_Size'].unique()
                 }

        fig = go.Figure(data=data, layout=layout)
        iplot(fig, show_link=False)